# Get code working

In [9]:
#This gets us into the right directory from home, in order to run the import python script from Mat

#sys = system (module), gives information and control over python interpreter/terminal itself
import sys
sys.path.append("/home/565/pv3484/aus_substation_electricity")

#% is a magic command, special shortcut command that lets you control/interact with notebook environment (ie. gives control of terminal without writing full python code)
%cd /home/565/pv3484/aus_substation_electricity

!pwd

/home/565/pv3484/aus_substation_electricity
/home/565/pv3484/aus_substation_electricity


In [10]:
#This section imports the substations that Mat put together

%run /home/565/pv3484/aus_substation_electricity/import_substation.py

processing nsw substations for ['ausgrid'] from None to None
ausgrid
following columns in demand are not in info index:
['MT_HU', 'SI_NO']
removing these columns from demand
number of substations in ausgrid substation info: 134
number of substations in ausgrid substation data: 132
following sites match selection criteria:
               energy_asset          Name  Area  Dwellings  Persons  Residential  Commercial  Industrial  Primary Production  Education  \
ID                                                                                                                                        
BLAKE         AG_BLAKEHURST    Blakehurst     7      10081    28521        0.850       0.005       0.021               0.000      0.022   
PUNCH          AG_PUNCHBOWL     Punchbowl     9      17514    50395        0.826       0.048       0.038               0.000      0.025   
MEADO         AG_MEADOWBANK    Meadowbank    15      22420    56948        0.825       0.023       0.015               0

# Soft Coding Demand with Residential Fraction for any chosen moving public holiday within chosen RF range
- Seperate from non moving holidays as this required different definition in function

In [21]:
import os
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

def plot_moving_holiday(demand, info,
                        holiday_dates,
                        holiday_name="Holiday",
                        rf_min=0.0, rf_max=0.2,
                        output_folder="./figures/MovingHolidays"):
    """
    Plot demand for a moving holiday (e.g. Good Friday) across all given dates,
    restricted to substations within a given Residential fraction range.
    """

    demand.index = pd.to_datetime(demand.index)

    # Restrict to substations within RF range
    rf_subset = info.index[(info['Residential'] >= rf_min) & (info['Residential'] <= rf_max)].tolist()
    substations = [s for s in demand.columns if s in rf_subset]

    # Determine years from holiday_dates
    years = sorted({d.year for d in holiday_dates})

    # Prepare colormap
    cmap = matplotlib.colormaps.get_cmap('plasma')
    norm = mcolors.Normalize(vmin=min(years), vmax=max(years))

    os.makedirs(output_folder, exist_ok=True)

    # Plotting demand
    for substation in substations:
        plt.figure(figsize=(10, 4))
        for holiday_date in holiday_dates:
            year = holiday_date.year
            day_data = demand[demand.index.date == holiday_date]
            if day_data.empty:
                continue

            x = day_data.index.hour + day_data.index.minute / 60.0
            y = day_data[substation].values

            color = cmap(norm(year))
            plt.plot(x, y, color=color, label=str(year))

        # Metadata
        full_name = info.loc[substation, 'Name']
        fraction = info.loc[substation, 'Residential']

        plt.title(f'Demand on {holiday_name} for {full_name} (Res {fraction:.2f})')
        plt.xlabel('Time of Day (hour)')
        plt.ylabel('Electricity Demand')
        plt.legend(title='Year', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize='small')
        plt.xlim(0, 24)
        xticks = np.arange(0, 25, 1)
        plt.xticks(xticks, [f'{int(t):02d}:00' for t in xticks], rotation=45)
        plt.tight_layout()

        # --- Save figure with holiday name + fraction ---
        filename = f"{full_name}_{holiday_name}.png"
        output_folder = "/home/565/pv3484/aus_substation_electricity/figures/Residential_fractions/GOODFRI/"
        filepath = os.path.join(output_folder, filename)
        plt.savefig(filepath, dpi=300)
        plt.close()

#change output_folder for specific folder for each pub hol

## How to use

In [23]:
#Good Friday

import holidays
aus_holidays = holidays.Australia(years=range(2010, 2016))
good_fridays = [date for date, name in aus_holidays.items() if name == "Good Friday"]

plot_moving_holiday(demand, info,
                    holiday_dates=good_fridays,
                    holiday_name="GoodFriday",
                    rf_min=0.4, rf_max=0.6,
                    output_folder=".")


#can change "Good Friday" to any moving public holiday name
#can change rf_min, rf_max for different RF ranges